In [1]:
import pandas as pd
import numpy as np

Data Exploration 

1. How many observations does your dataset have?

In [2]:
# Load the dataset files to count observations
import os

# Check what data files are available
data_files = [f for f in os.listdir('Data') if f.endswith('.csv')]
print("Available CSV files:")
for file in data_files:
    print(f"  - {file}")

print("\n" + "="*60)
print("DATASET OBSERVATION COUNTS")
print("="*60)

total_observations = 0

# Count observations in each file
for file in data_files:
    df = pd.read_csv(f'Data/{file}')
    rows, cols = df.shape
    total_observations += rows
    
    print(f"\n{file}:")
    print(f"  • Observations (rows): {rows:,}")
    print(f"  • Features (columns): {cols}")
    
    # Show what an observation represents
    if 'weekly' in file.lower():
        print(f"  • Each observation = one player's weekly stats")
    elif 'yearly' in file.lower():
        print(f"  • Each observation = one player's season stats")
    
    # Show a sample of the data structure
    print(f"  • Sample columns: {list(df.columns[:5])}")

print("\n" + "="*60)
print(f"TOTAL OBSERVATIONS ACROSS ALL FILES: {total_observations:,}")
print("="*60)

Available CSV files:
  - weekly_player_stats_defense.csv
  - weekly_player_stats_offense.csv
  - weekly_team_stats_defense.csv
  - weekly_team_stats_offense.csv
  - yearly_player_stats_defense.csv
  - yearly_player_stats_offense.csv
  - yearly_team_stats_defense.csv
  - yearly_team_stats_offense.csv

DATASET OBSERVATION COUNTS

weekly_player_stats_defense.csv:
  • Observations (rows): 117,993
  • Features (columns): 116
  • Each observation = one player's weekly stats
  • Sample columns: ['player_id', 'player_name', 'position', 'birth_year', 'draft_year']

weekly_player_stats_offense.csv:
  • Observations (rows): 58,629
  • Features (columns): 393
  • Each observation = one player's weekly stats
  • Sample columns: ['season', 'week', 'offense_snaps', 'offense_pct', 'team_offense_snaps']

weekly_team_stats_defense.csv:
  • Observations (rows): 7,088
  • Features (columns): 65
  • Each observation = one player's weekly stats
  • Sample columns: ['game_id', 'season', 'week', 'team', 'seas

2. Describe all columns in your dataset their scales and data distributions. Describe the categorical and continuous variables in your dataset. Describe your target column and if you are using images plot some example classes of the images.

3. Do you have missing and duplicate values in your dataset?

Data Plots

1. Plot your data with various types of charts like bar charts, pie charts, scatter plots etc. and clearly explain the plots.

2. For image data, you will need to plot your example classes.

How will you preprocess your data? Handle data imbalance if needed. You should only explain (do not perform pre-processing as that is in MS3) this in your README.md file and link your Jupyter notebook to it. All code and  Jupyter notebooks have be uploaded to your repo. (3 points)